AQUÍ COMENÇA EL CODI PER GENERAR UNA CALCULADORA QUE M'AJUDARÀ A TENIR UNA MILLOR ESTABILITAT FINANCERA

In [15]:
import matplotlib.pyplot as plt
from datetime import date, datetime




In [ ]:
saldo = 0.0 #Saldo inicial

import json
import os

registre_diari = {} #Diccionari per emmagatzemar els moviments

if os.path.exists("registre_despeses.json"):
    with open("registre_despeses.json", "r", encoding="utf-8") as f:
        dades = json.load(f)
        saldo = dades.get("saldo", 0.0)
        registre_diari = dades.get("registre", {})

def data_actual():
    return date.today().isoformat() # Retorna la data actual en format ISO (YYYY-MM-DD)

# Funció per registrar un moviment en el registre diari
def registrar_moviment(data, tipus_moviment, categoria, quantitat):
    if data not in registre_diari:
        registre_diari[data] = {
            "ingressos": [],
            "despeses": []
        }

    registre_diari[data][tipus_moviment].append({
        "categoria": categoria,
        "quantitat": quantitat
    })


ingressos = {
    "treball": 0.0,
    "dividends": 0.0,
    "interessos": 0.0,
    "regal": 0.0,
    "altres": 0.0
}

despeses = {
    "lloguer": 0.0,
    "alimentació": 0.0,
    "subministraments": 0.0,
    "transport": 0.0,
    "despeses llar": 0.0,
    "ETF": 0.0,
    "accions": 0.0,
    "viatges": 0.0,
    "oci": 0.0,
    "restaurant": 0.0,
    "altres": 0.0

}

# Crear carpeta si no existeix
os.makedirs("grafics", exist_ok=True)

# Només guardar si avui és dilluns (0 = dilluns)
es_dilluns = datetime.today().weekday() == 0
es_1r_dia_mes = datetime.today().day == 1
data_dia = datetime.today().strftime("%Y-%m-%d")



#Funció per demanar una quantitat numèrica i no peti si l'usuari introdueix un valor no numèric

def demanar_quantitat(missatge):
    while True:
        entrada = input(missatge)
        try:
            return float(entrada)
        except ValueError:
            print("⚠️  Si us plau, introdueix un número vàlid.")

while True:
    moviment=input("Has fet una despesa o has tingut un ingrés?")

    if moviment.lower() == "ingrés":

        data= data_actual()
        
        quantitat = demanar_quantitat("Quina quantitat has ingressat? ")
        tipus=input("Quin tipus d'ingrés és?")

        registrar_moviment(data, "ingressos", tipus.lower(), quantitat)

        saldo+=quantitat

        if tipus.lower() in ingressos:
            ingressos[tipus.lower()] += quantitat
        else:
            print("Tipus desconegut d'ingrés. S'està registrant com a 'altres'.")
            ingressos["altres"] += quantitat
        
        print(f"Has ingressat: {quantitat:.2f}€ en {tipus}.") 

    elif moviment == "despesa":

        data = data_actual()

        quantitat = demanar_quantitat("Quina quantitat has gastat? ")
        tipus=input("Quin tipus de despesa és?")

        registrar_moviment(data, "despeses", tipus.lower(), quantitat)

        saldo-=quantitat

        if tipus.lower() in despeses:
            despeses[tipus.lower()] += quantitat
        else:
            print("Tipus desconegut de despesa. S'està registrant com a 'altres'.")
            despeses["altres"] += quantitat
        print(f"Has gastat: {quantitat:.2f}€ en {tipus}.")
        
    elif moviment =="sortir":

        print("---Ingressos per tipus---")
        for tipus, quantitat in ingressos.items():
            print(f"{tipus.capitalize()}: {quantitat:.2f}€")

        #Calcular ingressos totals
        ingressos_totals = sum(ingressos.values())
        print(f"\nIngressos totals: {ingressos_totals:.2f}€")

        #Calcular rendiments de capital
        capital = ingressos.get("dividends", 0.0) + ingressos.get("interessos", 0.0)
        print(f"\nRendiments de capital (Dividends + Interessos): {capital:.2f}€")

        print("---Despeses per tipus---")
        for tipus, quantitat in despeses.items():
            print(f"{tipus.capitalize()}: {quantitat:.2f}€")

        #Calcular despeses totals
        despeses_totals = sum(despeses.values())
        print(f"\nDespeses totals: {despeses_totals:.2f}€")

        #Calcular despesa en inversió
        inversio = despeses.get("ETF", 0.0) + despeses.get("accions", 0.0)
        print(f"\nDespesa en inversió (ETF + Accions): {inversio:.2f}€")

        #Calcular despesa essencials
        essencials = despeses.get("lloguer", 0.0) + despeses.get("alimentació", 0.0) + despeses.get("subministraments", 0.0) + despeses.get("transport", 0.0) + despeses.get("despeses llar", 0.0)
        print(f"Despesa essencials (Lloguer + Alimentació): {essencials:.2f}€")

        #Calcular despesa en oci
        oci = despeses.get("viatges", 0.0) + despeses.get("oci", 0.0) + despeses.get("restaurant", 0.0)
        print(f"Despesa en oci (Viatges + Oci): {oci:.2f}€")

        #Calcular saldo final
        saldo_final = saldo
        print(f"\nSaldo final: {saldo_final:.2f}€")

        # Calcular totals
        total_ingressos = sum(ingressos.values())
        total_despeses = sum(despeses.values())

        import json
        with open("registre_despeses.json", "w", encoding="utf-8") as f:
            json.dump({
                "saldo":saldo,
                "registre": registre_diari,
            }, f, ensure_ascii=False, indent=4)

        # -------- GRÀFIC DE BARRA: Comparativa d'ingressos i despeses --------
        categories = ['Ingressos', 'Despeses']
        valors = [total_ingressos, total_despeses]
        if es_dilluns or es_1r_dia_mes:
            plt.bar(categories, valors, color=['green', 'red'])
            plt.title('Comparativa d\'ingressos i despeses')
            plt.ylabel('Euros (€)')
            plt.grid(axis='y')
            plt.savefig(f"grafics/comparativa_ingressos_despeses_{data_dia}.png")
            plt.close()
        

        # -------- GRÀFIC DE BARRA: Ingressos i despeses per dia --------

        #Llistes per constiruir el gràfic
        dates = []
        totals_ingressos = []
        totals_despeses = []

        # Recorrer el registre per data

        for data in sorted(registre_diari):
            dia=registre_diari[data]

            total_ingressos = sum(mov["quantitat"] for mov in dia["ingressos"])
            total_despeses = sum(mov["quantitat"] for mov in dia["despeses"])

            dates.append(data)
            totals_ingressos.append(total_ingressos)
            totals_despeses.append(total_despeses)

        #Crear el gràfic

        x= range(len(dates))

        if es_dilluns or es_1r_dia_mes:

            plt.figure(figsize=(10, 5))
            plt.bar(x, totals_ingressos, width=0.4, label='Ingressos', color='green', align='center')
            plt.bar([i + 0.4 for i in x], totals_despeses, width=0.4, label='Despeses', color='red', align='center')
            plt.xticks([i + 0.2 for i in x], dates, rotation=45)
            plt.ylabel("Quantitat (€)")
            plt.title("Ingressos i Despeses per Dia")
            plt.legend()
            plt.tight_layout()  # Ajusta el disseny per evitar solapaments
            plt.grid(axis='y')
            plt.savefig(f"grafics/barres_dia_{data_dia}.png")
            plt.close()

       



        # -------- GRÀFIC DE FORMATGE: Ingressos per categoria --------
        labels_ingressos = []
        valors_ingressos = []

        for categoria, quantitat in ingressos.items():
            if quantitat > 0:
                labels_ingressos.append(categoria.capitalize())
                valors_ingressos.append(quantitat)

        if (es_dilluns or es_1r_dia_mes) and valors_ingressos:
            plt.figure(figsize=(6, 6))
            plt.pie(valors_ingressos, labels=labels_ingressos, autopct='%1.1f%%', startangle=140)
            plt.title("Distribució dels Ingressos per Categoria")
            plt.axis('equal')  # Assegura que el gràfic sigui rodó
            plt.savefig(f"grafics/formatge_ingressos_{data_dia}.png")
            plt.close()
        

        # -------- GRÀFIC DE FORMATGE: Despeses per categoria --------
        labels_despeses = []
        valors_despeses = []

        for categoria, quantitat in despeses.items():
            if quantitat > 0:
                labels_despeses.append(categoria.capitalize())
                valors_despeses.append(quantitat)

        if (es_dilluns or es_1r_dia_mes) and valors_ingressos:
            plt.figure(figsize=(6, 6))
            plt.pie(valors_despeses, labels=labels_despeses, autopct='%1.1f%%', startangle=140)
            plt.title("Distribució de les Despeses per Categoria")
            plt.axis('equal')
            plt.savefig(f"grafics/formatge_despeses_{data_dia}.png")
            plt.close()


        break
    else:
        print("Moviment no vàlid. Si us plau, introdueix 'ingrés', 'despesa' o 'Sortir'.")
    
    



Has ingressat: 1500.00€ en treball.
Has gastat: 400.00€ en lloguer.
Has gastat: 50.00€ en oci.
Has gastat: 200.00€ en accions.
---Ingressos per tipus---
Treball: 1500.00€
Dividends: 0.00€
Interessos: 0.00€
Regal: 0.00€
Altres: 0.00€

Ingressos totals: 1500.00€

Rendiments de capital (Dividends + Interessos): 0.00€
---Despeses per tipus---
Lloguer: 400.00€
Alimentació: 0.00€
Subministraments: 0.00€
Transport: 0.00€
Despeses llar: 0.00€
Etf: 0.00€
Accions: 200.00€
Viatges: 0.00€
Oci: 50.00€
Restaurant: 0.00€
Altres: 0.00€

Despeses totals: 650.00€

Despesa en inversió (ETF + Accions): 200.00€
Despesa essencials (Lloguer + Alimentació): 400.00€
Despesa en oci (Viatges + Oci): 50.00€

Saldo final: 1975.00€


In [6]:
registre_diari

{'2025-07-09': {'ingressos': [{'categoria': 'treball', 'quantitat': 1500.0}],
  'despeses': [{'categoria': 'oci', 'quantitat': 300.0}]}}